In [43]:
# importing libraries 
import numpy as np
from attention_utils import get_data # scirpt to genetate data
import pandas as pd
#from __future__ import print_function
# import keras
import keras
#keras.__version__
print(keras.__version__)
from keras.layers import Dense
from keras.models import Sequential
from keras.models import Model
# all modules of keras for lSTM-RNNs
from keras.models import Sequential
from keras.layers import LSTM
#from keras.layers import Merge # problem with Merge because I now use Keras 2.2.4. Merge is replaced by add
from keras.layers import add
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop,Adamax
from keras import backend as K


2.2.4


In [44]:
# Reading files
# features
X_test=pd.read_csv('Test_set_1L_AC_aud_feat_win5_lag1_gsr.csv', sep=',',header=None) # reading ok #X_test.values
X_train=pd.read_csv('Train_set_1L_AC_aud_feat_win5_lag1_gsr.csv', sep=',',header=None)
X_valid=pd.read_csv('Valid_set_1L_AC_aud_feat_win5_lag1_gsr.csv', sep=',',header=None)
# labels
Y_test=pd.read_csv('Test_set_synch_feat_win5_lag1_Awe.csv', sep=',',header=None)
Y_train=pd.read_csv('Train_set_synch_feat_win5_lag1_Awe.csv', sep=',',header=None)
Y_valid=pd.read_csv('Valid_set_synch_feat_win5_lag1_Awe.csv', sep=',',header=None)
# small sets
X_test_s=X_test[0:101]
X_train_s=X_train[0:201]
X_valid_s=X_valid[0:101]
Y_test_s=Y_test[0:101]
Y_train_s=Y_train[0:201]
Y_valid_s=Y_valid[0:101]

In [45]:
n_features = X_train.shape[1] # columns are features , rows are observations
print(n_features)
input_dim = n_features
n_observation = X_train.shape[0] # columns are features , rows are observations
print(n_observation)

1100
19428


In [46]:
# define variables
nb_features = n_features # total number of features, also the number of neurons in the input layer of LSTM

time_step = 3  # the length of history (number of previous data instances) to include
batch_size = 32 # the model performs a weight update for every batch, smaller batch means faster learning but les stable weights
nb_epoch = 1000 # number of total epochs to train the model
H1 = 128 # number of neurons in the bottom hidden layer
H2 = 64 # number of neurons in the middle hidden layer
H3 = 32 # number of neurons in the top hidden layer
dropout_W1 = 0.5 # drop out weight (for preventing over-fitting) in H1
dropout_U1 = 0.5 # drop out weight (for preventing over-fitting) in H1
dropout_W2 = 0 # drop out weight (for preventing over-fitting) in H2
dropout_U2 = 0 # drop out weight (for preventing over-fitting) in H2
dropout_W3 = 0 # drop out weight (for preventing over-fitting) in H3
dropout_U3 = 0 # drop out weight (for preventing over-fitting) in H3

#opt_func = RMSprop(lr=0.0001) # training function
opt_func = Adamax(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

# if the validation loss isn't decreasing for a number of epochs, stop training to prevent over-fitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [47]:
# function to reshape the panda.DataFrame format data to Keras style: (batch_size, time_step, nb_features)
def reshape_data(data, n_prev = time_step):
    docX = []
    for i in range(len(data)):
        if i < (len(data)-n_prev):
            docX.append(data.iloc[i:i+n_prev].as_matrix())
        else: # the frames in the last window use the same context
            docX.append(data.iloc[(len(data)-n_prev):len(data)].as_matrix())
    alsX = np.array(docX)
    return alsX

In [48]:
# Reshaping the panda.DataFrame format data to Keras style: (batch_size, time_step, nb_features)
# shape (nb_observations, nb_features)
#X_Test = reshape_data(X_test)
#X_Train = reshape_data(X_train)
#X_Valid = reshape_data(X_valid)

# small set
X_Test = reshape_data(X_test_s)
X_Train = reshape_data(X_train_s)
X_Valid = reshape_data(X_valid_s)
# after reshaping: shape (nb_observations,time_step,nb_features)

In [49]:
# custom evaluation metrics
def pearson_cc(y_true, y_pred):
    fsp = y_pred - K.mean(y_pred,axis=0)   
    fst = y_true - K.mean(y_true,axis=0) 
    devP = K.std(y_pred,axis=0)  
    devT = K.std(y_true,axis=0)

    return K.sum(K.mean(fsp*fst,axis=0)/(devP*devT))

In [50]:
# turn off the warnings, be careful when use this
import warnings
warnings.filterwarnings("ignore")

In [51]:
# define model structure
model = Sequential()
model.add(LSTM(H1, input_shape=(time_step, nb_features),dropout_W=dropout_W1, dropout_U=dropout_U1, return_sequences=True)) 
# bottom hidden layer
model.add(LSTM(H2, dropout_W=dropout_W2, dropout_U=dropout_U2, return_sequences=True)) # middle hidden layer
model.add(LSTM(H3, dropout_W=dropout_W3, dropout_U=dropout_U3, return_sequences=False)) # top hidden layer
model.add(Dense(1, activation='sigmoid')) # output layer, regression task, value range [-1,1]

In [52]:
# Compiling
model.compile(loss=pearson_cc, optimizer=opt_func, metrics=[pearson_cc,'mse']) 
# define the optimizer for training, use cc and mse as the evaluation metric

In [53]:
# carry out training
model.fit(X_Train, Y_train_s, batch_size=batch_size, nb_epoch=nb_epoch, validation_data=(X_Valid, Y_valid_s), callbacks=[early_stopping], verbose=0)

In [54]:
# evaluation
test_score, test_cc, test_mse = model.evaluate(X_Test, Y_test_s, batch_size=batch_size, verbose=0)
print('Test cc:', test_cc)
print('Test mse:', test_mse)
print('\n')

Test cc: 0.19169994510046326
Test mse: 0.009705001489480607




In [55]:
# output predictions
test_pred = model.predict(X_Test)
test_df = pd.DataFrame(test_pred)
test_df.to_csv('prediction.csv', mode='a', index=False, header=False)